In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, DecimalType, DateType

# Configuração do Ambiente
catalogo = "medalhao"
silver = "silver"
gold = "gold"

# Define o catálogo e cria o schema Gold
spark.sql(f"USE CATALOG {catalogo}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {gold}")
spark.sql(f"USE SCHEMA {gold}")

print(f"Ambiente configurado: Catálogo '{catalogo}', Schema '{gold}'")

In [0]:
# Criação da Tabela Fato: gold.ft_vendas_consumidor_local
# Relacionando vendas com a localização do consumidor para otimização logística.

df_pedidos_total = spark.table(f"{catalogo}.{silver}.ft_pedidos_total")
df_consumidores = spark.table(f"{catalogo}.{silver}.ft_consumidores")

df_gold1 = (
    df_pedidos_total.alias("p")
    .join(df_consumidores.alias("c"), "id_consumidor")
    .select(
        F.col("p.id_pedido"),
        F.col("p.id_consumidor"),
        F.col("p.valor_total_pago_brl").alias("valor_total_pedido_brl"),
        F.col("p.valor_total_pago_usd"),
        F.col("c.cidade"),
        F.col("c.estado"),
        F.col("p.data_pedido")
    )
)

tabela_gold1 = f"{catalogo}.{gold}.ft_vendas_consumidor_local"
df_gold1.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(tabela_gold1)
print(f"Tabela criada: {tabela_gold1}")
df_gold1.printSchema()

In [0]:
# Criação da View Analítica: gold.view_total_compras_por_consumidor
spark.sql(f"""
    CREATE OR REPLACE VIEW {catalogo}.{gold}.view_total_compras_por_consumidor AS
    SELECT 
        cidade,
        estado,
        COUNT(*) AS quantidade_vendas,
        SUM(valor_total_pedido_brl) AS valor_total_localidade
    FROM {catalogo}.{gold}.ft_vendas_consumidor_local
    GROUP BY cidade, estado
""")
print("View criada: view_total_compras_por_consumidor")

# Consulta de Negócio
print(">> Executando consulta: Total de vendas por estado")
spark.sql(f"""
    SELECT estado, SUM(valor_total_localidade) AS total_vendas_brl
    FROM {catalogo}.{gold}.view_total_compras_por_consumidor
    GROUP BY estado
    ORDER BY total_vendas_brl DESC
""").display()

In [0]:
# Criação da Tabela Fato: gold.ft_atrasos_pedidos_local_vendedor
# Identificando gargalos logísticos por região e vendedor.

df_pedidos = spark.table(f"{catalogo}.{silver}.ft_pedidos")
df_itens = spark.table(f"{catalogo}.{silver}.ft_itens_pedidos")
df_consumidores = spark.table(f"{catalogo}.{silver}.ft_consumidores")

df_gold2 = (
    df_pedidos.alias("p")
    .join(df_consumidores.alias("c"), "id_consumidor")
    # Join com itens é necessário para trazer o vendedor
    .join(df_itens.alias("i"), "id_pedido") 
    .select(
        F.col("p.id_pedido"),
        F.col("i.id_vendedor"),
        F.col("p.id_consumidor"),
        F.col("p.entrega_no_prazo"),
        F.col("p.tempo_entrega_dias"),
        F.col("p.tempo_entrega_estimado_dias"),
        F.col("c.cidade"),
        F.col("c.estado")
    )
)

tabela_gold2 = f"{catalogo}.{gold}.ft_atrasos_pedidos_local_vendedor"
df_gold2.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(tabela_gold2)
print(f"Tabela criada: {tabela_gold2}")
df_gold2.printSchema()

In [0]:
# View: Tempo Médio de Entrega por Localidade
spark.sql(f"""
    CREATE OR REPLACE VIEW {catalogo}.{gold}.view_tempo_medio_entrega_localidade AS
    SELECT 
        cidade,
        estado,
        AVG(tempo_entrega_dias) AS tempo_medio_entrega,
        AVG(tempo_entrega_estimado_dias) AS tempo_medio_estimado,
        CASE 
            WHEN AVG(tempo_entrega_dias) > AVG(tempo_entrega_estimado_dias) THEN 'SIM' 
            ELSE 'NÃO'
        END AS entrega_maior_que_estimado
    FROM {catalogo}.{gold}.ft_atrasos_pedidos_local_vendedor
    GROUP BY cidade, estado
""")
print("View criada: view_tempo_medio_entrega_localidade")

In [0]:
# View: Pontualidade do Vendedor
spark.sql(f"""
    CREATE OR REPLACE VIEW {catalogo}.{gold}.view_vendedor_pontualidade AS
    SELECT 
        id_vendedor,
        COUNT(*) AS total_pedidos,
        SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) AS total_atrasados,
        (SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) / COUNT(*)) * 100 AS percentual_atraso
    FROM {catalogo}.{gold}.ft_atrasos_pedidos_local_vendedor
    GROUP BY id_vendedor
""")
print("View criada: view_vendedor_pontualidade")

In [0]:
# Criação da Dimensão Tempo: gold.dm_tempo
# Facilitando análises temporais (drill-down) permitindo visualização do macro para o micro

df_tempo = (
    spark.range(1)
    .select(F.explode(F.sequence(
        F.to_date(F.lit("2016-01-01")),
        F.to_date(F.lit("2018-12-31")),
        F.expr("INTERVAL 1 DAY")
    )).alias("sk_tempo"))
    .withColumn("ano", F.year("sk_tempo"))
    .withColumn("trimestre", F.quarter("sk_tempo"))
    .withColumn("mes", F.month("sk_tempo"))
    .withColumn("mes_nome", F.date_format("sk_tempo", "MMMM")) # Nome do mês
    .withColumn("semana_do_ano", F.weekofyear("sk_tempo"))
    .withColumn("dia", F.dayofmonth("sk_tempo"))
    .withColumn("dia_da_semana_num", F.dayofweek("sk_tempo"))
    .withColumn("dia_da_semana_nome", F.date_format("sk_tempo", "EEEE")) # Nome do dia
    .withColumn(
        "eh_fim_de_semana", 
        F.when(F.dayofweek("sk_tempo").isin(1, 7), "Sim").otherwise("Não") # 1=Dom, 7=Sab no Spark padrão
    )
)

tabela_tempo = f"{catalogo}.{gold}.dm_tempo"
df_tempo.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(tabela_tempo)
print(f"Tabela criada: {tabela_tempo}")
df_tempo.printSchema()

In [0]:
# Criação da Fato Geral: gold.ft_vendas_geral
# Tabela central para BI, granularidade no nível do Item.

df_itens = spark.table(f"{catalogo}.{silver}.ft_itens_pedidos")
df_pedidos = spark.table(f"{catalogo}.{silver}.ft_pedidos")
df_produtos = spark.table(f"{catalogo}.{silver}.ft_produtos")
df_vendedores = spark.table(f"{catalogo}.{silver}.ft_vendedores")
df_consumidores = spark.table(f"{catalogo}.{silver}.ft_consumidores")
df_dolar = spark.table(f"{catalogo}.{silver}.dm_cotacao_dolar")
df_avaliacoes = spark.table(f"{catalogo}.silver.ft_avaliacoes_pedidos") # Necessário para avaliação média
# Definição da Janela para preencher cotação faltante (fins de semana)
window_ffill = Window.orderBy("p.pedido_compra_timestamp").rowsBetween(Window.unboundedPreceding, 0)

df_gold3 = (
    df_itens.alias("i")
    # Joins usando strings fundem as colunas chave
    .join(df_pedidos.alias("p"), "id_pedido")
    .join(df_produtos.alias("pr"), "id_produto")
    .join(df_vendedores.alias("v"), "id_vendedor")
    .join(df_consumidores.alias("c"), "id_consumidor")
    # Join explícito (data) mantém os aliases
    .join(df_dolar.alias("d"), F.to_date(F.col("p.pedido_compra_timestamp")) == F.col("d.data"), "left")
    .join(df_avaliacoes.alias("a"), "id_pedido", "left")
    # Correção do Dólar pra finais de semana
    .withColumn("cotacao_final", F.last("d.cotacao_dolar", ignorenulls=True).over(window_ffill))
    .select(
        F.col("id_pedido"),
        F.col("i.id_item"),
        F.col("id_consumidor").alias("fk_cliente"),
        F.col("id_produto").alias("fk_produto"),
        F.col("id_vendedor").alias("fk_vendedor"),
        F.to_date(F.col("p.pedido_compra_timestamp")).alias("fk_tempo"),
        F.col("p.status").alias("status_pedido"),
        F.col("p.tempo_entrega_dias"),
        F.col("p.entrega_no_prazo"),
        F.col("i.preco_brl").alias("valor_produto_brl"),
        F.col("i.preco_frete").alias("valor_frete_brl"),
        (F.col("i.preco_brl") + F.col("i.preco_frete")).alias("valor_total_item_brl"),
        (F.col("i.preco_brl") / F.col("cotacao_final")).cast("decimal(12,2)").alias("valor_produto_usd"),
        (F.col("i.preco_frete") / F.col("cotacao_final")).cast("decimal(12,2)").alias("valor_frete_usd"),
        ((F.col("i.preco_brl") + F.col("i.preco_frete")) / F.col("cotacao_final")).cast("decimal(12,2)").alias("valor_total_item_usd"),
        F.col("cotacao_final").alias("cotacao_dolar"),
        F.col("a.avaliacao").alias("avaliacao_pedido")
    )
)

tabela_gold3 = f"{catalogo}.{gold}.ft_vendas_geral"
df_gold3.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(tabela_gold3)
print(f"Tabela criada com sucesso: {tabela_gold3}")

df_gold3.printSchema()

In [0]:
# View: gold.view_vendas_por_periodo
spark.sql(f"""
    CREATE OR REPLACE VIEW {catalogo}.{gold}.view_vendas_por_periodo AS
    SELECT 
        t.ano,
        t.trimestre,
        t.mes,
        t.mes_nome,
        t.dia,
        t.dia_da_semana_num,
        t.dia_da_semana_nome,
        COUNT(DISTINCT f.id_pedido) AS total_pedidos,
        COUNT(f.id_item) AS total_itens,
        SUM(f.valor_total_item_brl) AS receita_total_brl,
        SUM(f.valor_total_item_usd) AS receita_total_usd,
        AVG(f.valor_total_item_brl) AS ticket_medio_brl,
        AVG(f.avaliacao_pedido) AS avaliacao_media
    FROM {catalogo}.{gold}.ft_vendas_geral f
    JOIN {catalogo}.{gold}.dm_tempo t ON f.fk_tempo = t.sk_tempo
    GROUP BY t.ano, t.trimestre, t.mes, t.mes_nome, t.dia, t.dia_da_semana_num, t.dia_da_semana_nome
""")
print("View criada: view_vendas_por_periodo")

In [0]:
# Consultas Analíticas
print(">> Executando consultas analíticas de BI...")

print("1. Dia da semana com maior receita total (BRL):")
spark.sql(f"""
    SELECT dia_da_semana_nome, SUM(receita_total_brl) as receita_total 
    FROM {catalogo}.{gold}.view_vendas_por_periodo 
    GROUP BY dia_da_semana_nome 
    ORDER BY receita_total DESC LIMIT 1
""").display()

print("2. Mês com maior ticket médio (BRL) no último ano:")
spark.sql(f"""
    SELECT mes_nome, ticket_medio_brl 
    FROM {catalogo}.{gold}.view_vendas_por_periodo 
    WHERE ano = (SELECT MAX(ano) FROM {catalogo}.{gold}.view_vendas_por_periodo) 
    ORDER BY ticket_medio_brl DESC LIMIT 1
""").display()

In [0]:
# View: Top Produto
spark.sql(f"""
    CREATE OR REPLACE VIEW {catalogo}.{gold}.view_top_produto AS
    SELECT
        f.fk_produto AS id_produto,
        p.categoria_produto,
        COUNT(*) AS quantidade_vendida,
        COUNT(DISTINCT f.id_pedido) AS total_pedidos,
        SUM(f.valor_total_item_brl) AS receita_brl,
        SUM(f.valor_total_item_usd) AS receita_usd,
        AVG(f.valor_produto_brl) AS preco_medio_brl,
        AVG(f.avaliacao_pedido) AS avaliacao_media,
        AVG(p.peso_produto_gramas) AS peso_medio_gramas
    FROM {catalogo}.{gold}.ft_vendas_geral f
    JOIN {catalogo}.{silver}.ft_produtos p ON f.fk_produto = p.id_produto
    GROUP BY f.fk_produto, p.categoria_produto
""")
print("View criada: view_top_produto")
# Check dos tipos
view_top_produto = spark.sql(f"SELECT * FROM {catalogo}.{gold}.view_top_produto")
display(view_top_produto)




In [0]:
# View: Vendas Produtos Estéticos (CTE)
spark.sql(f"""
    CREATE OR REPLACE VIEW {catalogo}.{gold}.view_vendas_produtos_esteticos AS
    WITH base AS (
        SELECT 
            f.id_pedido,
            f.id_item,
            f.fk_produto,
            f.valor_total_item_brl,
            f.valor_total_item_usd,
            f.avaliacao_pedido,
            t.ano,
            t.mes
        FROM {catalogo}.{gold}.ft_vendas_geral f
        JOIN {catalogo}.{gold}.dm_tempo t
            ON f.fk_tempo = t.sk_tempo
        JOIN {catalogo}.{silver}.ft_produtos p
            ON f.fk_produto = p.id_produto
        WHERE p.categoria_produto LIKE 'fashion%'
    )
    SELECT 
        ano,
        mes,
        'Fashion' as categoria_produto,
        COUNT(DISTINCT id_pedido) AS total_pedidos,
        COUNT(*) AS total_itens_vendidos,
        SUM(valor_total_item_brl) AS receita_total_brl,
        SUM(valor_total_item_usd) AS receita_total_usd,
        AVG(valor_total_item_brl) AS ticket_medio_brl,
        AVG(valor_total_item_usd) AS ticket_medio_usd,
        AVG(avaliacao_pedido) AS avaliacao_media
    FROM base
    GROUP BY ano, mes
""")
print("View criada: view_vendas_produtos_esteticos")